<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E4%B8%89%E9%80%B1_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [48]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [49]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').sheet1

In [68]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1zrCPUzVIOPlAc7P3ZXT_TcfVAyXl1qO8n-gIeuOg38A/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [51]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [52]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [53]:
df.describe()

,Account,Date,Cleaned_Content
count,312,312,312
unique,272,13,312
top,peculab,"Sep 5, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
freq,11,110,1


In [42]:
!pip install jieba
!pip install snownlp

In [54]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [73]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [74]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [77]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=False)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [78]:
word_counts = Counter(all_words)
word_counts

Counter({'姓名': 2,
         '以及': 3,
         '留下': 1,
         '第一': 9,
         '程式': 157,
         '想要': 29,
         '心中': 1,
         '想法': 1,
         '高中生': 3,
         '助教': 7,
         '三年': 1,
         '大便': 1,
         '小孩子': 1,
         '大家': 145,
         '整理': 2,
         '可以': 54,
         '一下': 6,
         '下面': 3,
         '那位': 1,
         '冷面': 1,
         '小心': 1,
         '不正': 2,
         '不想': 1,
         '直接': 1,
         '失效': 1,
         '上面': 2,
         '各位': 6,
         '大三': 1,
         '一名': 1,
         '歌手': 1,
         '可惜': 1,
         '一些': 6,
         '因素': 1,
         '不了': 2,
         '一天': 3,
         '收到': 1,
         '高手': 3,
         '感到': 3,
         '好奇': 3,
         '密密麻麻': 2,
         '一堆': 1,
         '看出': 1,
         '到底': 1,
         '如何': 2,
         '不信邪': 1,
         '想到': 2,
         '奇怪': 1,
         '踏入': 1,
         '以上': 2,
         '如有雷同': 1,
         '巧合': 1,
         '研究': 4,
         '助理': 2,
         '如此': 2,
         '自己': 

In [79]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [80]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [32]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-wqy-zenhei is already the newest version (0.9.45-8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [81]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [62]:
# 按日期計算討論熱度
df['Date'] = pd.to_datetime(df['Date'])
heatmap_data = df.groupby('Date').size().reset_index(name='Discussion_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='Discussion_Count',
              title='不同時間段的討論熱度趨勢',
              labels={'Date': '日期', 'Discussion_Count': '討論數量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="討論數量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [63]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [64]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，以及提到這個意思的人是誰：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

以下摘要留言重點，包含情緒、發言者：


**學生留言 (大部分展現積極、好奇、期待學習的情緒):**

* **對程式設計/AI有興趣:**  大多數學生表達對程式設計或AI的濃厚興趣，想學習相關知識，並應用於未來生活、學習或工作。(多數學生，例如：7, 8, 9, 10, 14, 15, 17, 18, 19, 20, 21, 22, 27, 29, 30, 32, 34, 46, 47, 48, 49, 50, 57, 61, 63, 64, 66, 67, 69, 70, 72, 74, 77, 78, 79, 80, 82, 83, 85, 86, 87, 88, 89, 90, 91, 94, 98, 99, 101, 102, 105, 107, 108, 109, 112, 113, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 202, 203, 205, 206, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 224, 227, 228, 229, 230, 234, 235, 236, 238, 240, 241, 252, 254, 255, 257, 259, 262, 263, 266, 267, 269, 270, 271, 272, 273, 274, 276, 277, 278, 279, 281, 282, 283, 